In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler

In [2]:
data_df = pd.read_excel('./data_df_full.xlsx')

In [3]:
data_df.head()

,이름,이미지,링크,가격,분류,리뷰건수,등록일,찜하기,급여대상,중량,주원료,등급,기능,입자크기
0,내추럴발란스 LID 고구마 연어 건식사료 작은입자,https://shopping-phinf.pstatic.net/main_144180...,https://cr.shopping.naver.com/adcr.nhn?x=r1RXI...,32000,건식사료,3250,2018.05.,101,어덜트,6120,"고구마, 연어",그레인프리,"영양공급, 피부/털개선, 저알러지, 면역력강화, 눈물개선/눈건강",작은 8mm 미만
1,ANF 램 홀리스틱 프리미엄 램28 15kg 10kg 양고기 애견사료,https://shopping-phinf.pstatic.net/main_819317...,https://cr.shopping.naver.com/adcr.nhn?x=M%2Bj...,49850,건식사료,1675,2021.09.,182,"어덜트, 대형견",15000,"양, 쌀",홀리스틱(1등급),"영양공급, 뼈/관절강화, 피부/털개선, 체중유지",보통 8~13mm
2,아스쿠 펠리쿠치나 1.2kg 3kg 반습식 강아지 사료 소고기 오리 연어,https://shopping-phinf.pstatic.net/main_815662...,https://cr.shopping.naver.com/adcr.nhn?x=RPCfu...,20000,소프트사료,1752,2019.01.,89,"어덜트, 시니어, 임신/수유, 대형견",1200,"소, 오리, 연어",그레인프리,영양공급,작은 8mm 미만
3,강아지사료 20kg 프라임 블랙에디션 25 34 퍼포먼스 애견 특수견 진도 대용량 ...,https://shopping-phinf.pstatic.net/main_821120...,https://cr.shopping.naver.com/adcr.nhn?x=T8bj4...,26500,건식사료,5660,2019.06.,277,"퍼피, 어덜트, 임신/수유, 대형견",20000,"닭, 곡물",프리미엄(3등급),"영양공급, 퍼포먼스",보통 8~13mm
4,하림 더리얼 그레인프리 크런치 오븐베이크드 모음전,https://shopping-phinf.pstatic.net/main_823923...,https://cr.shopping.naver.com/adcr.nhn?x=ms%2B...,17100,건식사료,9436,2020.03.,401,"퍼피, 어덜트, 시니어, 임신/수유",1000,"닭, 양, 연어",그레인프리,"영양공급, 피부/털개선, 저알러지, 인도어, 식욕증진(기호성)",보통 8~13mm


# 유사도 가중합산

In [4]:
# 복수값을 허용하는 열에 대해 one-hot-encoding
def feat_to_vec(data_df, col):
    items = set(",".join(data_df[col]).replace(" ","").split(','))
    data_df_col = pd.DataFrame(columns = list(items))

    for i in range(len(data_df[col])):
        new_tuple = dict(zip(list(items), [0]*len(list(items)))) 
        for item_have in data_df[col][i].replace(" ","").split(','):
            for item_candidate in list(items):
                if item_have == item_candidate:
                    new_tuple[item_candidate]=1
        data_df_col = data_df_col.append(new_tuple, ignore_index=True)
        
    return data_df_col


# 토큰화 및 불용어 제거한 corpus 생성

def make_cleaned_corpus(str):
    tokenizer = Okt()
    del_list = ['kg'] 
    raw_pos_tagged = tokenizer.pos(str, norm=True, stem=True)
    word_cleaned = []
    for word in raw_pos_tagged:
        if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign", "Number"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
            if word[0] not in del_list: # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
                word_cleaned.append(word[0])
    result = " ".join(word_cleaned)
    return result


cleaned_name = []
for i in range(len(data_df['이름'])):
    word_cleaned = make_cleaned_corpus(data_df['이름'][i])
    cleaned_name.append(word_cleaned)
    
# 이름열 기반으로 tfidf_vectorizer fitting 

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(cleaned_name) # 벡터라이저가 단어들을 학습합니다.

tfidf_matrix = tfidf_vectorizer.transform(cleaned_name).todense()

data_df_func = feat_to_vec(data_df,'기능')
data_df_target = feat_to_vec(data_df,'급여대상')
data_df_ingredient = feat_to_vec(data_df,'주원료')
data_df_cat = feat_to_vec(data_df,'분류')
data_df_grade =  feat_to_vec(data_df,'등급')

for i in range(len(data_df_target)):
    if data_df_target['전연령'][i]==1:
        data_df_target['시니어'][i]=1
        data_df_target['퍼피'][i]=1
        data_df_target['어덜트'][i]=1           
        
data_df_target = data_df_target.drop('전연령', axis=1)


In [5]:
!pip install dill

In [6]:
import dill as pickle

In [7]:
pickle.dump(tfidf_vectorizer, open("tfidf_vectorizer.pkl", "wb"))

In [9]:
def get_score_on_col(query, data_df_col):
    cleaned_query = make_cleaned_corpus(query)
    querys = cleaned_query.split(' ')
    items = list(data_df_col.columns)
    query_tuple = dict(zip(items, [0]*len(items)))
    
    for query in querys:
        for item in items:
            if query in item:
                query_tuple[item]=1 # 두번 입력해도 가중 X
    
    scores = []
    for i in range(len(data_df_col)):
        score = 0
        for item in items:
            if query_tuple[item]==1 & data_df_col[item][i]==1:
                score +=1
        scores.append(score)
        
        scaler = MinMaxScaler()
        result = scaler.fit_transform(np.array(scores).reshape(len(scores),1))
        result = result.reshape(len(result))
    return result

In [10]:
def dogfood_search(query):
    cleaned_query = make_cleaned_corpus(query)
    query_tfidf = tfidf_vectorizer.transform([cleaned_query]).todense()

    name_cosine_sim = cosine_similarity(query_tfidf, tfidf_matrix)[0]
    func_scores = get_score_on_col(query, data_df_func)
    target_scores = get_score_on_col(query, data_df_target)
    ingredient_scores = get_score_on_col(query, data_df_ingredient)
    cat_scores = get_score_on_col(query, data_df_cat)
    grade_scores = get_score_on_col(query, data_df_grade)


    a = 3 # 이름 가중치
    b = 1 # 기능 가중치
    c = 1 # 급여대상 가중치
    d = 1 # 주원료 가중치
    e = 1 # 분류 가중치
    f = 1 # 등급 가중치
    total_score =(a*name_cosine_sim+b*func_scores+c*target_scores+d*ingredient_scores+e*cat_scores+f*grade_scores)/(a+b+c+d+e+f)
    result = pd.concat([data_df, pd.DataFrame(total_score, columns = ['score'])], axis = 1)
    result = result.sort_values('score',ascending=False)
    
    return result
    

In [11]:
dogfood_search("그레인프리 대형견 사료")

,이름,이미지,링크,가격,분류,리뷰건수,등록일,찜하기,급여대상,중량,주원료,등급,기능,입자크기,score
593,나우사료 그레인프리 올브리드 어덜트 9.98kg 중대형견 강아지사료,https://shopping-phinf.pstatic.net/main_836319...,https://cr.shopping.naver.com/adcr.nhn?x=0Vwhm...,116100,건식사료,2,2022.03.,2,"어덜트, 대형견",9980,"오리, 칠면조, 연어",그레인프리,"영양공급, 피부/털개선, 소화/장기능개선",보통 8~13mm,0.519746
142,요세라 강아지사료 4.5 kg 중형견 대형견 그레인프리,https://shopping-phinf.pstatic.net/main_838252...,https://cr.shopping.naver.com/adcr.nhn?x=xWFIs...,71000,건식사료,2,2022.02.,8,"어덜트, 시니어, 대형견",4500,"과일/야채, 기타",그레인프리,"치석제거, 영양공급, 뼈/관절강화, 피부/털개선, 저알러지",보통 8~13mm,0.470149
475,이볼브 그레인프리 독 연어 5.44kg,https://shopping-phinf.pstatic.net/main_823016...,https://cr.shopping.naver.com/adcr.nhn?x=%2BUx...,57000,건식사료,49,2019.12.,1,"퍼피, 어덜트, 시니어, 대형견",5440,연어,그레인프리,"피부/털개선, 눈물개선/눈건강, 식욕증진(기호성)",작은 8mm 미만,0.395432
119,웰썸 인섹트 관절 사료 3.6kg 그레인프리 강아지 눈물사료,https://shopping-phinf.pstatic.net/main_829885...,https://cr.shopping.naver.com/adcr.nhn?x=PKmPT...,59400,건식사료,146,2021.03.,14,"퍼피, 어덜트, 시니어, 임신/수유, 대형견",3600,기타,그레인프리,영양공급,작은 8mm 미만,0.385920
494,웰썸 인섹트 관절 사료 3.6kg 그레인프리 강아지 눈물사료,https://shopping-phinf.pstatic.net/main_829885...,https://cr.shopping.naver.com/adcr.nhn?x=%2BWY...,59400,건식사료,146,2021.03.,1,"퍼피, 어덜트, 시니어, 임신/수유, 대형견",3600,기타,그레인프리,영양공급,작은 8mm 미만,0.385920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,힐스 사이언스다이어트 강아지 라이트 스몰바이트 어덜트7+ 스몰포 6.8kg 7kg,https://shopping-phinf.pstatic.net/main_834507...,https://cr.shopping.naver.com/adcr.nhn?x=wAxwC...,69900,건식사료,12,2022.03.,11,어덜트,6800,"닭, 과일/야채",홀리스틱(1등급),"영양공급, 피부/털개선",보통 8~13mm,0.000000
55,내추럴발란스 LID 닭고기 고구마 도그 포뮬라,https://shopping-phinf.pstatic.net/main_221669...,https://cr.shopping.naver.com/adcr.nhn?x=tWsPo...,55350,건식사료,764,2020.03.,14,어덜트,6120,닭,홀리스틱(1등급),"영양공급, 피부/털개선, 저알러지, 면역력강화, 소화/장기능개선",작은 8mm 미만,0.000000
788,ANF 6Free PLUS 플러스 양고기 5.6kg,https://shopping-phinf.pstatic.net/main_859087...,https://cr.shopping.naver.com/adcr.nhn?x=pruGX...,37500,건식사료,354,2022.04.,1,"전연령, 퍼피, 어덜트",5600,양,유기농,영양공급,작은 8mm 미만,0.000000
37,웰츠 독 저지방 다이어트,https://shopping-phinf.pstatic.net/main_210479...,https://cr.shopping.naver.com/adcr.nhn?x=Xmmb5...,17000,건식사료,425,2019.10.,24,"퍼피, 어덜트, 시니어",2100,닭,홀리스틱(1등급),"영양공급, 피부/털개선, 다이어트/중성화, 면역력강화",보통 8~13mm,0.000000
